In [1]:
import pandas as pd
import time
from openai import OpenAI

# [초기 설정] API 키와 클라이언트 설정
# 여기에 본인의 OpenAI API Key를 입력하세요.
API_KEY = "api key"
client = OpenAI(api_key=API_KEY)

def generate_hard_negatives():
    """
    연구 논문의 방법론에 따라 4가지 괴롭힘 클래스에 대응하는 
    하드 네거티브 일반 대화(Class 04) 1,000개를 생성합니다.
    """
    
    # 1단계: 도메인 분석 및 시나리오 설계 [cite: 70]
    # 각 괴롭힘 유형(00~03)에 대응하는 하드 네거티브 시나리오 정의
    scenarios = {
        "00_협박대응": {
            "target": "협박(00)",
            "desc": "층간소음, 주차 문제 등에 대해 법적 절차와 경찰 신고를 언급하는 정당하고 단호한 경고 [cite: 47]",
            "count": 250
        },
        "01_갈취대응": {
            "target": "갈취(01)",
            "desc": "친구 사이의 소액 빌리기, 중고 거래 흥정, 생일 선물 제안 등 상호 호혜적 금전 대화 [cite: 50]",
            "count": 250
        },
        "02_직장대응": {
            "target": "직장 내 괴롭힘(02)",
            "desc": "인격 모독 없이 전문적인 비즈니스 가이드라인에 따른 엄격한 업무 피드백 및 수정 지시 [cite: 52]",
            "count": 250
        },
        "03_기타대응": {
            "target": "기타 괴롭힘(03)",
            "desc": "외모나 신체를 언급하되 발화자 간의 정서적 지지와 공통된 웃음이 포함된 친밀한 장난 [cite: 54]",
        }
    }

    generated_data = []

    for key, info in scenarios.items():
        print(f"--- {key} 시나리오 생성 시작 (목표: {info['count']}개) ---")
        
        for i in range(info['count']):
            # 2, 3단계: 의도 사슬(CoI) 및 스타일 대조 학습(SCL) 프롬프트 적용 [cite: 21, 38]
            system_prompt = f"""
            당신은 한국어 괴롭힘 분류 모델의 성능 향상을 위한 '하드 네거티브' 일반 대화 생성기입니다.
            목표: {info['target']}과 어휘적으로 유사하지만, 실제로는 무해한 '일반 대화(04)'를 생성하세요.
            
            [지침]
            1. 의도 사슬(CoI): 대화는 [상황 발생 -> 의견 교환 -> 상세 설명 -> 합리적 수용]의 흐름을 가져야 합니다. [cite: 28]
            2. 스타일(SCL): 실제 한국어 구어체(감탄사, '~하네요', '~했나 봐요')를 사용하여 현실성을 높이세요. 
            3. 맥락: {info['desc']}
            4. 제약: 인격 모독이나 위법적 행위는 절대 포함하지 마세요.
            
            형식: 5턴 이상의 대화문만 출력하세요. (예: 인물A: 내용 / 인물B: 내용)
            """
            
            try:
                response = client.chat.completions.create(
                    model="gpt-4",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": "새로운 하드 네거티브 일반 대화 샘플을 하나 생성해줘."}
                    ],
                    temperature=0.8 # 다양성을 위한 설정 
                )
                
                conv_text = response.choices[0].message.content.strip()
                generated_data.append({"class": "일반 대화", "conversation": conv_text})
                
                # API 호출 간격 조절 (Rate Limit 방지)
                if (i + 1) % 10 == 0:
                    print(f"{key}: {i+1}/{info['count']} 완료...")
                    time.sleep(1)
                    
            except Exception as e:
                print(f"오류 발생: {e}")
                continue

    return pd.DataFrame(generated_data)

# 4단계: 데이터 통합 및 모델 재학습 준비 
def merge_and_save(new_df, original_path='train.csv'):
    try:
        # 1. 기존 데이터 로드 (00~03 클래스)
        original_train = pd.read_csv(original_path)
        
        # 2. 데이터 병합
        # 기존 데이터와 생성된 1,000개의 일반 대화 데이터를 합칩니다.
        total_train = pd.concat([original_train, new_df], ignore_index=True)
        
        # 3. 데이터 셔플 (순서 섞기)
        # 특정 클래스가 몰려 있지 않도록 무작위로 섞습니다.
        total_train = total_train.sample(frac=1).reset_index(drop=True)
        
        # 4. 최종 결과 저장
        save_name = 'total_train_with_normal.csv'
        total_train.to_csv(save_name, index=False, encoding='utf-8-sig')
        print(f"성공: {len(total_train)}개의 데이터가 {save_name}에 저장되었습니다.")
        
    except FileNotFoundError:
        print("오류: train.csv 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

# --- 실행부 ---
if __name__ == "__main__":
    # 1. 일반 대화 데이터 1,000개 생성 (시간이 다소 소요될 수 있습니다)
    normal_conversations_df = generate_hard_negatives()
    
    # 2. 기존 train.csv와 병합 및 저장
    merge_and_save(normal_conversations_df)

--- 00_협박대응 시나리오 생성 시작 (목표: 250개) ---
00_협박대응: 10/250 완료...
00_협박대응: 20/250 완료...
00_협박대응: 30/250 완료...
00_협박대응: 40/250 완료...
00_협박대응: 50/250 완료...
00_협박대응: 60/250 완료...
00_협박대응: 70/250 완료...
00_협박대응: 80/250 완료...
00_협박대응: 90/250 완료...
00_협박대응: 100/250 완료...
00_협박대응: 110/250 완료...
00_협박대응: 120/250 완료...
00_협박대응: 130/250 완료...
00_협박대응: 140/250 완료...
00_협박대응: 150/250 완료...
00_협박대응: 160/250 완료...
00_협박대응: 170/250 완료...


KeyboardInterrupt: 

# Feature 2. Hard negative

In [2]:
import pandas as pd
import time
from openai import OpenAI

# [초기 설정] API 키와 클라이언트 설정
API_KEY = "api key"
client = OpenAI(api_key=API_KEY)

def generate_hard_negatives():
    """
    연구 논문의 방법론에 따라 하드 네거티브 일반 대화(Class 04)를 생성합니다.
    중간에 정지해도 그때까지 생성된 데이터를 반환합니다. [cite: 9, 70]
    """
    
    scenarios = {
        "00_협박대응": {
            "target": "협박(00)",
            "desc": "층간소음, 주차 문제 등에 대해 법적 절차와 경찰 신고를 언급하는 정당하고 단호한 경고 [cite: 47]",
            "count": 250
        },
        "01_갈취대응": {
            "target": "갈취(01)",
            "desc": "친구 사이의 소액 빌리기, 중고 거래 흥정, 생일 선물 제안 등 상호 호혜적 금전 대화 [cite: 50]",
            "count": 250
        },
        "02_직장대응": {
            "target": "직장 내 괴롭힘(02)",
            "desc": "인격 모독 없이 전문적인 비즈니스 가이드라인에 따른 엄격한 업무 피드백 및 수정 지시 [cite: 52]",
            "count": 250
        },
        "03_기타대응": {
            "target": "기타 괴롭힘(03)",
            "desc": "외모나 신체를 언급하되 발화자 간의 정서적 지지와 공통된 웃음이 포함된 친밀한 장난 [cite: 54]",
            "count": 250
        }
    }

    generated_data = []

    try:
        for key, info in scenarios.items():
            print(f"--- {key} 시나리오 생성 시작 (목표: {info['count']}개) ---")
            
            for i in range(info['count']):
                # 모델을 gpt-4o-mini로 변경하여 비용 절감 
                system_prompt = f"""
                당신은 한국어 괴롭힘 분류 모델의 성능 향상을 위한 '하드 네거티브' 일반 대화 생성기입니다.
                목표: {info['target']}과 어휘적으로 유사하지만, 실제로는 무해한 '일반 대화(04)'를 생성하세요. [cite: 14]
                
                [지침]
                1. 의도 사슬(CoI): [상황 발생 -> 의견 교환 -> 상세 설명 -> 합리적 수용] 흐름 준수 [cite: 28]
                2. 스타일(SCL): 실제 한국어 구어체(감탄사, '~하네요', '~했나 봐요') 활용 [cite: 40]
                3. 맥락: {info['desc']}
                4. 제약: 인격 모독이나 위법적 행위 절대 금지 [cite: 52]
                
                형식: 5턴 이상의 대화문만 출력 (예: 인물A: 내용 / 인물B: 내용)
                """
                
                try:
                    response = client.chat.completions.create(
                        model="gpt-4o-mini", # 비용 효율적인 모델로 변경
                        messages=[
                            {"role": "system", "content": system_prompt},
                            {"role": "user", "content": "새로운 하드 네거티브 일반 대화 샘플을 하나 생성해줘."}
                        ],
                        temperature=0.8
                    )
                    
                    conv_text = response.choices[0].message.content.strip()
                    generated_data.append({"class": "일반 대화", "conversation": conv_text})
                    
                    if (i + 1) % 10 == 0:
                        print(f"{key}: {i+1}/{info['count']} 완료... (현재 총 {len(generated_data)}개)")
                        time.sleep(0.5)
                        
                except Exception as e:
                    print(f"API 오류 발생: {e}")
                    continue

    except KeyboardInterrupt:
        # 주피터 노트북에서 '정지' 버튼을 누르면 이 구간이 실행됩니다.
        print("\n🛑 사용자에 의해 생성이 중단되었습니다. 현재까지 생성된 데이터를 저장합니다...")

    # 중단되더라도 생성된 만큼은 데이터프레임으로 변환하여 반환
    return pd.DataFrame(generated_data)

def merge_and_save(new_df, original_path='train.csv'):
    if new_df.empty:
        print("생성된 데이터가 없어 병합을 중단합니다.")
        return

    try:
        original_train = pd.read_csv(original_path)
        total_train = pd.concat([original_train, new_df], ignore_index=True)
        total_train = total_train.sample(frac=1).reset_index(drop=True)
        
        save_name = 'total_train_with_normal.csv'
        total_train.to_csv(save_name, index=False, encoding='utf-8-sig')
        print(f"✅ 최종 성공: 총 {len(total_train)}개의 데이터(새 데이터 {len(new_df)}개 포함)가 {save_name}에 저장되었습니다.")
        
    except FileNotFoundError:
        # train.csv가 없을 경우 생성된 데이터라도 따로 저장
        new_df.to_csv('generated_normal_only.csv', index=False, encoding='utf-8-sig')
        print(f"⚠️ train.csv를 찾을 수 없어 생성된 {len(new_df)}개의 데이터만 별도로 저장했습니다.")

if __name__ == "__main__":
    normal_conversations_df = generate_hard_negatives()
    merge_and_save(normal_conversations_df)

--- 00_협박대응 시나리오 생성 시작 (목표: 250개) ---
00_협박대응: 10/250 완료... (현재 총 10개)
00_협박대응: 20/250 완료... (현재 총 20개)
00_협박대응: 30/250 완료... (현재 총 30개)
00_협박대응: 40/250 완료... (현재 총 40개)
00_협박대응: 50/250 완료... (현재 총 50개)
00_협박대응: 60/250 완료... (현재 총 60개)
00_협박대응: 70/250 완료... (현재 총 70개)
00_협박대응: 80/250 완료... (현재 총 80개)
00_협박대응: 90/250 완료... (현재 총 90개)
00_협박대응: 100/250 완료... (현재 총 100개)
00_협박대응: 110/250 완료... (현재 총 110개)
00_협박대응: 120/250 완료... (현재 총 120개)
00_협박대응: 130/250 완료... (현재 총 130개)
00_협박대응: 140/250 완료... (현재 총 140개)
00_협박대응: 150/250 완료... (현재 총 150개)
00_협박대응: 160/250 완료... (현재 총 160개)
00_협박대응: 170/250 완료... (현재 총 170개)
00_협박대응: 180/250 완료... (현재 총 180개)
00_협박대응: 190/250 완료... (현재 총 190개)
00_협박대응: 200/250 완료... (현재 총 200개)
00_협박대응: 210/250 완료... (현재 총 210개)
00_협박대응: 220/250 완료... (현재 총 220개)
00_협박대응: 230/250 완료... (현재 총 230개)
00_협박대응: 240/250 완료... (현재 총 240개)
00_협박대응: 250/250 완료... (현재 총 250개)
--- 01_갈취대응 시나리오 생성 시작 (목표: 250개) ---
01_갈취대응: 10/250 완료... (현재 총 260개)
01_갈취대응: 20/250 완료... (현재 총 270개)

In [3]:
import pandas as pd
import re

def clean_speaker_labels(text):
    """
    문장 시작 부분의 '화자명: ' 패턴을 제거합니다.
    예: '인물A: 안녕하세요' -> '안녕하세요'
    """
    if not isinstance(text, str):
        return text
    
    # 정규표현식 설명:
    # ^: 줄의 시작
    # [^:\n]+: 콜론(:)이나 줄바꿈이 아닌 문자들이 하나 이상 나옴 (화자 이름)
    # :\s*: 콜론과 그 뒤의 공백문자
    cleaned_text = re.sub(r'^[^:\n]+:\s*', '', text, flags=re.MULTILINE)
    
    # 각 줄의 앞뒤 불필요한 공백 제거
    lines = [line.strip() for line in cleaned_text.split('\n')]
    return '\n'.join(lines)

# 1. 데이터 로드
df = pd.read_csv('total_train_with_normal.csv')

# 2. '일반 대화' 클래스에만 정제 함수 적용
# 기존 클래스(00~03)는 건드리지 않고 새로 만든 데이터만 수정합니다.
df.loc[df['class'] == '일반 대화', 'conversation'] = df.loc[df['class'] == '일반 대화', 'conversation'].apply(clean_speaker_labels)

# 3. 결과 확인 (샘플 출력)
print("--- 정제된 일반 대화 예시 ---")
print(df[df['class'] == '일반 대화']['conversation'].iloc[0])

# 4. 최종 통합 데이터 저장
df.to_csv('total_train_final.csv', index=False, encoding='utf-8-sig')
print("\n✅ 모든 클래스의 형식이 일치된 'total_train_final.csv' 파일이 생성되었습니다.")

--- 정제된 일반 대화 예시 ---
요즘 너 살 좀 빠진 것 같아, 운동 열심히 했나?
아, 진짜? 그런 거 같기도 해! 그냥 매일 걸어다니려고 노력하고 있어.
그러게, 요즘 좋은 생각이네. 나도 좀 따라 해야겠다. 나 저번에 체중계 올라갔는데 깜짝 놀랐어!
뭐야, 얼마야? 너무 궁금하잖아!
비밀이야! 그냥 살짝만 늘었지 뭐. 너처럼 운동 열심히 해야겠다, 진짜!
하하, 괜찮아. 우리 같이 운동하자! 그러면 서로 동기 부여도 되고 재밌을 것 같아.
좋지! 그러면서 서로의 운동복 패션도 비교해보자!

✅ 모든 클래스의 형식이 일치된 'total_train_final.csv' 파일이 생성되었습니다.


In [5]:
!pip install openai pandas scikit-learn


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.0 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/8.0 MB 2.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.0/8.0 MB 2.6 MB/s eta 0:00:03
   ------ --------------------------------- 1.3/8.0 MB 1.4 MB/s eta 0:00:05
   ------- -------------------------------- 1.6/8.0 MB 1.4 MB/s eta 0:00:05
   ------------- -------------------------- 2.6/8.0 MB 2.0 MB/s eta 0:00:03
   -------------------- ------------------- 4.2/8.0 MB 2.9 MB/s eta 0:00:02
   -------------------------------- ------- 6.6/8.0 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 4.4 MB/s  0:00:01
   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
   -- ------------------------------------- 1.8/36.3 MB 10.7 MB/s eta 0:00:04
   -- ------------------------------------- 2.4/36.3 MB 5.5 MB/s eta 0:00:07
   -- ------------------

# Feature 3.Augmentation을 통한 데이터 증강 및 LLM 판사도입을 통한 일반 대화 데이터 편집

In [7]:
import pandas as pd
import time
import random
import re
import json
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# [초기 설정] API 키 입력
API_KEY = "api key"
client = OpenAI(api_key=API_KEY)

# ==========================================
# 1. 헬퍼 함수 (유사도 계산, LLM 호출)
# ==========================================

def calculate_similarity(new_text, existing_texts, vectorizer):
    """
    새로운 텍스트와 기존 텍스트들 간의 코사인 유사도를 계산합니다.
    최대 유사도가 0.95를 넘으면 True(중복)를 반환합니다.
    """
    if not existing_texts:
        return False, 0.0
    
    try:
        # 벡터화 (기존 텍스트 리스트에 새 텍스트를 추가하여 변환)
        tfidf_matrix = vectorizer.transform(existing_texts + [new_text])
        # 마지막 행(새 텍스트)과 나머지 행들(기존) 간의 유사도 계산
        cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
        max_sim = cosine_sim.max()
        return max_sim > 0.95, max_sim
    except:
        return False, 0.0

def call_llm(messages, model="gpt-4o-mini", temperature=0.7):
    """OpenAI API 호출 래퍼 함수 (오류 처리 포함)"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"API Error: {e}")
        time.sleep(1) # 오류 시 잠시 대기
        return None

# ==========================================
# 2. 핵심 기능: 데이터 증강 (Augmentation) - 1,000개 목표
# ==========================================

def augment_existing_data(df, target_count=1000):
    """
    기존 train.csv의 각 클래스(00~03) 데이터를 '재작성(Paraphrasing)'하여 증강합니다.
    목표: 클래스별 1,000개 추가 생성
    """
    print(f"\n🚀 기존 데이터 증강 시작 (각 클래스별 {target_count}개 생성 목표)...")
    augmented_list = []
    
    # 클래스별로 데이터 분리
    classes = df['class'].unique()
    
    for cls in classes:
        if cls == '일반 대화': continue 
        
        class_data = df[df['class'] == cls]['conversation'].tolist()
        if not class_data: continue

        print(f"--- Class '{cls}' 증강 중 ---")
        
        for i in range(target_count):
            # 랜덤하게 원본 샘플 선택 (복원 추출)
            original_sample = random.choice(class_data)
            
            prompt = f"""
            당신은 데이터 증강 전문가입니다. 아래 대화의 '핵심 의도'와 '클래스({cls})' 특성은 유지하되, 
            문장 스타일, 단어, 상황을 조금 바꿔서 새로운 대화 데이터를 하나 만들어주세요.
            
            [원본 대화]
            {original_sample}
            
            [요청사항]
            1. 원본과 의미는 같지만 표현을 다르게 하세요 (예: 사투리, 줄임말, 더 격한 표현 등).
            2. 길이는 원본과 비슷하게 유지하세요.
            3. 결과물에는 오직 '대화 내용'만 출력하세요.
            """
            
            new_conv = call_llm([{"role": "user", "content": prompt}], temperature=0.9)
            
            if new_conv:
                augmented_list.append({'class': cls, 'conversation': new_conv})
            
            if (i + 1) % 50 == 0:
                print(f"   {i+1}/{target_count} 완료...")
                
    return pd.DataFrame(augmented_list)

# ==========================================
# 3. 핵심 기능: 하드 네거티브 생성 - 2,000개 목표 (500 * 4)
# ==========================================

def generate_hard_negatives_advanced(existing_df):
    
    # 3-1. 시나리오별 500개씩 설정 (총 2,000개)
    scenarios = {
        "00_협박대응": {
            "target": "협박(00)",
            "topics": ["층간소음 항의", "주차 시비", "중고차 사기 대응", "악성 댓글 고소 예고", "계약 불이행 내용증명", "기물 파손 배상 요구", "임금 체불 항의", "반려동물 소음 갈등"],
            "count": 500 
        },
        "01_갈취대응": {
            "target": "갈취(01)",
            "topics": ["친구에게 돈 빌리기", "더치페이 정산 재촉", "중고 거래 네고(흥정)", "회비 걷기", "빌려준 물건 돌려받기", "생일 선물 사달라고 조르기", "여행 경비 정산", "식대 선결제 요청"],
            "count": 500
        },
        "02_직장대응": {
            "target": "직장 내 괴롭힘(02)",
            "topics": ["마감 기한 재촉", "보고서 전면 수정 지시", "근태(지각) 지적", "성과 평가 면담", "업무 실수에 대한 경위서 요구", "복장 규정 안내", "휴가 일정 조정 요청", "회의 태도 지적"],
            "count": 500
        },
        "03_기타대응": {
            "target": "기타 괴롭힘(03)",
            "topics": ["절친 간의 외모 디스(장난)", "패션 지적과 조언", "사투리/말투 흉내내기", "게임 실력 놀리기", "과거 흑역사 언급하며 웃기", "연애사 캐묻기(친한 사이)", "노래 실력 타박하기"],
            "count": 500
        }
    }

    # 3-2. 유사도 필터링을 위한 Vectorizer 준비
    print("\n📊 유사도 분석을 위한 데이터 벡터화 중...")
    vectorizer = TfidfVectorizer(max_features=5000)
    
    # 기존 데이터가 있으면 fit 수행 (없으면 임시 텍스트로)
    all_texts = existing_df['conversation'].tolist() if not existing_df.empty else ["임시 텍스트"]
    vectorizer.fit(all_texts)
    
    generated_data = []
    
    try:
        for key, info in scenarios.items():
            print(f"\n--- {key} 시나리오 생성 시작 (목표: {info['count']}개) ---")
            success_count = 0
            
            while success_count < info['count']:
                # [Diversity] 세부 토픽 랜덤 선택 (쏠림 방지)
                topic = random.choice(info['topics'])
                
                # 1. 생성 (Generation)
                gen_prompt = f"""
                목표: '{info['target']}'과 혼동되기 쉽지만 무해한 '일반 대화(04)' 생성.
                상황: {topic}
                
                [지침]
                1. 의도 사슬(CoI): 상황 발생 -> 항의/요구 -> 설명 -> 수용
                2. 스타일: 한국어 구어체, 감탄사 포함
                3. 제약: 욕설, 인격 모독 절대 금지. 법적/사회적으로 정당한 범위 내의 대화.
                
                형식: 5턴 이상의 대화만 출력. 화자 구분(A:, B:) 포함.
                """
                
                conv_text = call_llm([{"role": "system", "content": gen_prompt}], temperature=0.85)
                if not conv_text: continue

                # 2. 판사 평가 (LLM-as-a-Judge)
                judge_prompt = f"""
                당신은 데이터 품질 판사입니다. 아래 대화가 '일반 대화'로서 적절한지 평가하세요.
                
                [대화]
                {conv_text}
                
                [기준]
                1. 안전성: 괴롭힘/범죄 요소가 없는가? (협박성 발언이 있어도 정당한 방어라면 통과)
                2. 명확성: {info['target']} 클래스와 헷갈리지만 결국 '일반 대화'임이 확실한가?
                
                답변 형식: JSON 포맷 {{ "score": 1~5점, "reason": "이유" }}
                5점이 만점, 3점 이하는 불합격.
                """
                judge_res = call_llm([{"role": "user", "content": judge_prompt}], temperature=0.1)
                
                # 점수 파싱
                score = 0
                if judge_res:
                    try:
                        import re
                        score_match = re.search(r'"score":\s*(\d)', judge_res)
                        if score_match: score = int(score_match.group(1))
                    except: pass
                
                # 3. 피드백 루프 (Feedback Loop) - 점수 낮으면 수정
                if score < 4:
                    # print(f"   ↪️ [재작성] 품질 미달(점수:{score}). 수정 중...") # 로그 너무 많으면 주석 처리
                    rewrite_prompt = f"""
                    판사 피드백: {judge_res}
                    위 피드백을 반영하여 대화를 더 자연스럽고 '일반 대화'스럽게 수정해줘.
                    """
                    conv_text = call_llm([
                        {"role": "user", "content": gen_prompt},
                        {"role": "assistant", "content": conv_text},
                        {"role": "user", "content": rewrite_prompt}
                    ])
                
                # 4. 유사도 필터링 (Similarity Check)
                is_duplicate, sim_score = calculate_similarity(conv_text, all_texts, vectorizer)
                if is_duplicate:
                    print(f"   🗑️ [삭제] 기존 데이터와 너무 유사함 (유사도: {sim_score:.2f})")
                    continue 

                # 통과! 저장
                generated_data.append({"class": "일반 대화", "conversation": conv_text})
                all_texts.append(conv_text) # 중복 검사용 리스트에 추가
                
                success_count += 1
                if success_count % 50 == 0:
                    print(f"   ✅ {key}: {success_count}/{info['count']} 생성 완료")

    except KeyboardInterrupt:
        print("\n🛑 사용자 중단! 현재까지의 데이터를 저장합니다.")
    
    return pd.DataFrame(generated_data)

# ==========================================
# 4. 메인 실행부
# ==========================================

def main():
    try:
        # 1. 기존 데이터 로드
        original_df = pd.read_csv('train.csv')
        print(f"📂 train.csv 로드 완료 ({len(original_df)}개)")
    except:
        print("⚠️ train.csv가 없습니다. 빈 데이터프레임으로 시작합니다.")
        original_df = pd.DataFrame(columns=['class', 'conversation'])

    # 2. 기존 데이터 증강 (Augmentation)
    # 클래스당 1,000개 추가 생성 -> 기존 1,000개 + 증강 1,000개 = 2,000개
    augmented_df = augment_existing_data(original_df, target_count=1000)
    
    # 3. 하드 네거티브 생성 (Generation)
    # 총 2,000개 생성 (시나리오별 500개)
    combined_for_check = pd.concat([original_df, augmented_df], ignore_index=True)
    normal_df = generate_hard_negatives_advanced(combined_for_check)
    
    # 4. 최종 통합
    final_df = pd.concat([original_df, augmented_df, normal_df], ignore_index=True)
    
    # 셔플 및 저장
    final_df = final_df.sample(frac=1).reset_index(drop=True)
    final_df.to_csv('final_augmented_train_2k.csv', index=False, encoding='utf-8-sig')
    print(f"\n🎉 모든 작업 완료! 총 {len(final_df)}개의 데이터가 'final_augmented_train_2k.csv'에 저장되었습니다.")
    
    # 클래스별 개수 확인 출력
    print("\n📊 최종 클래스별 데이터 개수:")
    print(final_df['class'].value_counts())

if __name__ == "__main__":
    main()

📂 train.csv 로드 완료 (3950개)

🚀 기존 데이터 증강 시작 (각 클래스별 1000개 생성 목표)...
--- Class '협박 대화' 증강 중 ---
   50/1000 완료...
   100/1000 완료...
   150/1000 완료...
   200/1000 완료...
   250/1000 완료...
   300/1000 완료...
   350/1000 완료...
   400/1000 완료...
   450/1000 완료...
   500/1000 완료...
   550/1000 완료...
   600/1000 완료...
   650/1000 완료...
   700/1000 완료...
   750/1000 완료...
   800/1000 완료...
   850/1000 완료...
   900/1000 완료...
   950/1000 완료...
   1000/1000 완료...
--- Class '기타 괴롭힘 대화' 증강 중 ---
   50/1000 완료...
   100/1000 완료...
   150/1000 완료...
   200/1000 완료...
   250/1000 완료...
   300/1000 완료...
   350/1000 완료...
   400/1000 완료...
   450/1000 완료...
   500/1000 완료...
   550/1000 완료...
   600/1000 완료...
   650/1000 완료...
   700/1000 완료...
   750/1000 완료...
   800/1000 완료...
   850/1000 완료...
   900/1000 완료...
   950/1000 완료...
   1000/1000 완료...
--- Class '갈취 대화' 증강 중 ---
   50/1000 완료...
   100/1000 완료...
   150/1000 완료...
   200/1000 완료...
   250/1000 완료...
   300/1000 완료...
   350/1000 완료...
   4

In [9]:
!pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]




[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
import time
import random
from openai import OpenAI

# [초기 설정] API 키 입력
API_KEY = "api key"
client = OpenAI(api_key=API_KEY)

def generate_custom_normal_to_excel(total_count=300):
    """
    사용자 정의 프롬프트를 기반으로 하드 네거티브 일반 대화 300개를 생성하고 엑셀로 저장합니다.
    """
    generated_data = []
    
    # 1. 길이 조건 분산 설정 (비율 20:20:10 -> 300개 기준 120:120:60) [cite: 41]
    # 150~200자(120개), 200~280자(120개), 280~350자(60개)
    len_buckets = (["150~200자"] * 120) + (["200~280자"] * 120) + (["280~350자"] * 60)
    random.shuffle(len_buckets)

    # 2. 다양성 확보를 위한 상황(Context) 예시 리스트
    # 주제당 15개를 넘지 않도록 다양한 상황을 프롬프트에 동적으로 주입합니다. [cite: 70]
    contexts = [
        "절친끼리 비속어를 섞은 거친 장난", "식당에서 정당한 환불 요구로 인한 실랑이", 
        "층간소음으로 인한 이웃 간의 격한 말다툼", "연인 간의 이별 통보와 감정적 다툼",
        "중고 거래 중 돈 문제로 인한 경찰 언급 시나리오", "친구에게 빌려준 돈을 받기 위한 재촉",
        "팀장님과의 격식 없는 사적인 술자리 대화", "가족 간의 사소한 오해로 시작된 큰 소리",
        "게임 중 팀원과의 거친 피드백 교환", "주차 문제로 인한 차주들 간의 실랑이"
    ]

    print(f"🚀 신규 일반 대화 {total_count}개 생성 및 엑셀 구축 시작...")

    try:
        for i in range(total_count):
            target_len = len_buckets[i]
            current_context = random.choice(contexts)
            
            # 사용자 제공 조건을 집약한 정교한 시스템 프롬프트 [cite: 44, 55]
            system_prompt = f"""
            당신은 한국어 대화 데이터 생성 전문가입니다. 다음 조건을 완벽히 만족하는 '일반 대화(04)'를 생성하세요.
            
            [지침]
            - 주제: {current_context} (부정적인 표현이나 비속어 장난이 섞여도 됨)
            - 성격: 자연스럽지만 부정적인 일상 대화. 단, 괴롭히는 뉘앙스(위계관계 압박 등)는 절대 없어야 함.
            - 필수 활용 가능 단어: 죽이, 살리, 내놔, 뒤지, 새끼, 만원, 협박, 갈취, 부장님, 돼지, 환불, 경찰, 신고, 살려주세요, 돈이 등
            - 길이: {target_len} 내외
            - 발화 횟수: 4~8턴
            - 형식: "{i+1}. (약 {target_len}자)"로 시작하고, 이후 화자 구분(A:, B:)을 포함한 대화문 출력
            """

            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini", # 비용 효율성 및 속도 최적화 [cite: 30]
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": "조건에 맞는 한국어 일상 대화 샘플을 하나 생성해줘."}
                    ],
                    temperature=0.9 # 다양성 확보를 위한 높은 창의성 설정 [cite: 41]
                )
                
                conv_text = response.choices[0].message.content.strip()
                generated_data.append({
                    "번호": i + 1,
                    "class": "일반 대화",
                    "conversation": conv_text,
                    "설정길이": target_len,
                    "상황분류": current_context
                })
                
                if (i + 1) % 10 == 0:
                    print(f"✅ {i+1}/{total_count}개 생성 중...")
                    
            except Exception as e:
                print(f"API 오류 발생 (인덱스 {i}): {e}")
                time.sleep(2)
                continue

    except KeyboardInterrupt:
        print("\n🛑 사용자에 의해 중단되었습니다. 현재까지의 데이터를 저장합니다.")

    # 3. 데이터프레임 변환 및 엑셀 저장
    if generated_data:
        new_df = pd.DataFrame(generated_data)
        
        # 엑셀 파일로 저장 (xlsx 파일을 위해 openpyxl 라이브러리가 필요할 수 있습니다)
        file_name = "custom_normal_dialogue_300.xlsx"
        new_df.to_excel(file_name, index=False, engine='openpyxl')
        
        print(f"\n✨ 생성 완료! '{file_name}' 파일이 저장되었습니다.")
        print(new_df.head())
    else:
        print("생성된 데이터가 없습니다.")

if __name__ == "__main__":
    generate_custom_normal_to_excel(300)

🚀 신규 일반 대화 300개 생성 및 엑셀 구축 시작...
✅ 10/300개 생성 중...
✅ 20/300개 생성 중...
✅ 30/300개 생성 중...
✅ 40/300개 생성 중...
✅ 50/300개 생성 중...
✅ 60/300개 생성 중...
✅ 70/300개 생성 중...
✅ 80/300개 생성 중...
✅ 90/300개 생성 중...
✅ 100/300개 생성 중...
✅ 110/300개 생성 중...
✅ 120/300개 생성 중...
✅ 130/300개 생성 중...
✅ 140/300개 생성 중...
✅ 150/300개 생성 중...
✅ 160/300개 생성 중...
✅ 170/300개 생성 중...
✅ 180/300개 생성 중...
✅ 190/300개 생성 중...
✅ 200/300개 생성 중...
✅ 210/300개 생성 중...
✅ 220/300개 생성 중...
✅ 230/300개 생성 중...
✅ 240/300개 생성 중...
✅ 250/300개 생성 중...
✅ 260/300개 생성 중...
✅ 270/300개 생성 중...
✅ 280/300개 생성 중...
✅ 290/300개 생성 중...
✅ 300/300개 생성 중...

✨ 생성 완료! 'custom_normal_dialogue_300.xlsx' 파일이 저장되었습니다.
   번호  class                                       conversation      설정길이  \
0   1  일반 대화  1. A: 야, 너 내가 만원 빌린 거 언제 갚을 거야? 진짜 죽이기 전에 내놔. ...  150~200자   
1   2  일반 대화  2. (약 200~280자)\n\nA: 야, 부장님 진짜 오늘 너무 이상해. 팀 회...  200~280자   
2   3  일반 대화  3. (약 280~350자)\n\nA: 진짜, 이번에도 또 밤새도록 신발 소리 왜 ...  280~350자   
3   4  일반 대화  4. (약 200~280자)\n\nA: 야, 이번

In [15]:
import pandas as pd
import re
import os

# [수정 포인트] 파일 이름을 확인하세요.
FILE_NAME = 'custom_normal_dialogue_300.csv'

def clean_conversation(text):
    """
    1. 번호 및 글자 수 정보 제거 (예: 1. (약 150자))
    2. 화자 식별자 제거 (A:, B: 등)
    """
    if not isinstance(text, str):
        return text
    
    # 1. 시작 번호와 (약 XXX자) 패턴 제거
    text = re.sub(r'^\d+\.\s*(\(약?\s*\d+~\d+자\))?\s*', '', text)
    
    # 2. 각 줄의 시작 부분에 있는 화자 이름(A:, 인물A: 등)과 콜론 제거
    text = re.sub(r'^[^:\n]+:\s*', '', text, flags=re.MULTILINE)
    
    # 3. 앞뒤 공백 제거 및 빈 줄 제외하고 다시 합치기
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    return '\n'.join(lines)

# 파일 읽기 및 정제 수행
try:
    if os.path.exists(FILE_NAME):
        # [핵심 수정] encoding='cp949'를 추가하여 한글 인코딩 오류 해결
        df = pd.read_csv(FILE_NAME, encoding='cp949')
        
        # 정제 함수 적용
        df['conversation'] = df['conversation'].apply(clean_conversation)
        
        # 4단계: 최종 저장 (엑셀에서도 잘 열리도록 utf-8-sig 사용) [cite: 74]
        output_name = "custom_normal_dialogue_300_cleaned.csv"
        df.to_csv(output_name, index=False, encoding='utf-8-sig')
        
        print(f"✨ 정제 성공! '{output_name}' 파일이 생성되었습니다.")
        print("\n--- 첫 번째 데이터 확인 ---")
        print(df['conversation'].iloc[0])
    else:
        print(f"❌ 파일을 찾을 수 없습니다: {FILE_NAME}")

except UnicodeDecodeError:
    # 혹시 CP949로도 안 될 경우 EUC-KR 시도
    df = pd.read_csv(FILE_NAME, encoding='euc-kr')
    print("💡 EUC-KR 인코딩으로 파일을 읽었습니다.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")

✨ 정제 성공! 'custom_normal_dialogue_300_cleaned.csv' 파일이 생성되었습니다.

--- 첫 번째 데이터 확인 ---
야, 너 내가 만원 빌린 거 언제 갚을 거야? 진짜 죽이기 전에 내놔.
뭔 소리야? 내가 언제 네한테 만원 빌렸다고? 새끼야, 진짜.
기억 안 나? 그날 너 자꾸 협박하더니 나한테 돈 갈취하려고 했잖아!
어? 그거 그냥 농담이었어! 경찰 신고하고 싶으면 해봐, 내가 살아남을게.
그러니까! 살려주세요, 내 돈이 문제야!
내가 돈 없으면 환불도 못 받아. 정말 뒤지기 싫다면 알아서 해!
